In [1]:
import os
os.chdir("../..")

from experiments.vae.vae import VAE, VAEConfig, set_seeds, RESOURCES, TBLogger
from experiments.data import MNISTDataset, train_val_dataset_split
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torch
from pathlib import Path

[experiments.base] [INFO] Added console handler
[experiments.base] [INFO] Resources path: /home/akashg/git/NNs-and-Deep-Learning/experiments/../resources
[experiments.base] [INFO] Torch device: cpu


In [6]:
set_seeds()

model_dir = "/home/akashg/git/NNs-and-Deep-Learning/resources/models/vae/20240807-222134"
model_dir = Path(model_dir)

model = VAE.load_model(Path(model_dir))

train_set, test_set = MNISTDataset()
train_set, val_set = train_val_dataset_split(train_set, val_size=10000)

val_loader = DataLoader(
    val_set,
    batch_size=model.config.batch_size,
    shuffle=False,
    collate_fn=lambda x: torch.stack([i[0] for i in x]),
)

# Run forward pass on validation set and save images
model = model.eval()

[experiments.base] [INFO] Train size: 60000
[experiments.base] [INFO] Test size: 10000
[experiments.base] [INFO] Train set: 50000
[experiments.base] [INFO] Valid set: 10000


In [9]:
val_fpath = model_dir / "val"
val_fpath.mkdir(exist_ok=True)

original_fpath = val_fpath / "original"
original_fpath.mkdir(exist_ok=True)

for i, x in enumerate(val_loader):
    x = x.to(model.device)
    x_hat, _, _ = model(x)
    x_hat = x_hat.view(-1, 1, 28, 28)
    save_image(x_hat, val_fpath / f"{i}.png")
    x = x.view(-1, 1, 28, 28)
    save_image(x, original_fpath / f"{i}.png")
    if i == 10:
        break
